<a href="https://colab.research.google.com/github/ProfeLuisTic1986/Tuplas/blob/main/Calabazas_brujas_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame, random, sys
pygame.init()

# ---------------- CONFIGURACIÓN BÁSICA ----------------
ANCHO, ALTO = 480, 640
TAM_CALABAZA = (40, 30)
TAM_BRUJA    = (32, 26)

# Colores
NOCHE  = (15, 15, 30)
LUNA   = (230, 230, 200)
NARANJA = (255, 140, 0)
BLANCO  = (255, 255, 255)
MORADO  = (120, 70, 200)
SOMBRA  = (60, 0, 70)
MARRON  = (120, 80, 40)

# Dificultades: clave → (vel_jugador, num_brujas, vel_min, vel_max)
DIFICULTADES = {
    "1": (5, 3, 2, 4),   # Fácil
    "2": (6, 4, 3, 7),   # Normal
    "3": (7, 6, 5, 9),   # Difícil
}

# Valores iniciales (modo Normal = "2")
VEL_JUGADOR, NUM_BRUJAS, VEL_MIN, VEL_MAX = DIFICULTADES["2"]

# ---------------- INICIALIZACIÓN PYGAME ----------------
pantalla = pygame.display.set_mode((ANCHO, ALTO))
pygame.display.set_caption("Calabaza vs Brujas")
reloj = pygame.time.Clock()
font = pygame.font.SysFont(None, 28)

# ---------------- FUNCIONES AUXILIARES ----------------
def rect_tupla(pos, tam):
    """Devuelve (x, y, w, h) a partir de dos tuplas"""
    return (*pos, *tam)

def crear_brujas():
    """Genera la lista de brujas según la dificultad actual."""
    return [
        [random.randint(0, ANCHO - TAM_BRUJA[0]), -TAM_BRUJA[1], random.randint(VEL_MIN, VEL_MAX)]
        for _ in range(NUM_BRUJAS)
    ]

def dibujar_calabaza(pos):
    x, y = pos
    w, h = TAM_CALABAZA
    # Cuerpo
    pygame.draw.ellipse(pantalla, NARANJA, (x, y, w, h))
    # Tallo
    pygame.draw.rect(pantalla, (20, 120, 20), (x + w//2 - 4, y - 6, 8, 8))

def dibujar_bruja(pos):
    x, y = pos
    w, h = TAM_BRUJA
    # Cuerpo
    pygame.draw.ellipse(pantalla, SOMBRA, (x, y+6, w, h-6))
    # Gorro
    pygame.draw.polygon(pantalla, SOMBRA, [
        (x + w//2,   y - 6),
        (x + w//2 - 10, y + 8),
        (x + w//2 + 10, y + 8)
    ])
    pygame.draw.rect(pantalla, SOMBRA, (x + w//2 - 12, y + 8, 24, 4))
    # Escoba
    pygame.draw.line(pantalla, MARRON, (x-6, y+h-6), (x+w+10, y+h-3), 3)
    pygame.draw.polygon(pantalla, MARRON, [
        (x+w+10, y+h-3),
        (x+w+18, y+h-9),
        (x+w+18, y+h+3)
    ])
    # Ojos
    pygame.draw.circle(pantalla, MORADO, (x + w//2 - 4, y + 12), 2)
    pygame.draw.circle(pantalla, MORADO, (x + w//2 + 4, y + 12), 2)

def reiniciar_juego():
    """Reinicia posición del jugador, brujas, puntaje y estado."""
    jugador = [ANCHO//2 - TAM_CALABAZA[0]//2, ALTO - 80]
    brujas = crear_brujas()
    return jugador, brujas, 0, False

# ---------------- ESTADO INICIAL ----------------
jugador_pos, brujas, puntaje, game_over = reiniciar_juego()

# ---------------- BUCLE PRINCIPAL ----------------
while True:
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if e.type == pygame.KEYDOWN:
            # Cambiar dificultad (1,2,3)
            if e.unicode in DIFICULTADES and not game_over:
                VEL_JUGADOR, NUM_BRUJAS, VEL_MIN, VEL_MAX = DIFICULTADES[e.unicode]
                jugador_pos, brujas, puntaje, game_over = reiniciar_juego()
            # Reiniciar cuando hay game over
            if game_over and e.key == pygame.K_r:
                jugador_pos, brujas, puntaje, game_over = reiniciar_juego()

    keys = pygame.key.get_pressed()

    if not game_over:
        # Movimiento del jugador
        dx = (keys[pygame.K_RIGHT] - keys[pygame.K_LEFT]) * VEL_JUGADOR
        dy = (keys[pygame.K_DOWN] - keys[pygame.K_UP]) * VEL_JUGADOR
        jugador_pos[0] += dx
        jugador_pos[1] += dy

        # Límites
        jugador_pos[0] = max(0, min(ANCHO - TAM_CALABAZA[0], jugador_pos[0]))
        jugador_pos[1] = max(0, min(ALTO  - TAM_CALABAZA[1], jugador_pos[1]))

        # Movimiento de brujas
        for b in brujas:
            b[1] += b[2]  # b = [x, y, vel]
            # Si sale por abajo, reaparece arriba y sumas punto
            if b[1] > ALTO:
                b[0] = random.randint(0, ANCHO - TAM_BRUJA[0])
                b[1] = -TAM_BRUJA[1]
                b[2] = random.randint(VEL_MIN, VEL_MAX)
                puntaje += 1

            # Colisión con el jugador
            rect_jugador = pygame.Rect(rect_tupla(jugador_pos, TAM_CALABAZA))
            rect_bruja   = pygame.Rect(b[0], b[1], *TAM_BRUJA)
            if rect_jugador.colliderect(rect_bruja):
                game_over = True

    # ---------------- DIBUJO ----------------
    pantalla.fill(NOCHE)
    pygame.draw.circle(pantalla, LUNA, (ANCHO-70, 70), 40)

    # Dibujar jugador y brujas
    dibujar_calabaza(jugador_pos)
    for b in brujas:
        dibujar_bruja((b[0], b[1]))

    # Texto en pantalla
    if not game_over:
        texto = f"1=Fácil  2=Normal  3=Difícil   Puntos: {puntaje}"
        color = BLANCO
    else:
        texto = "¡Te atraparon!  R = reiniciar"
        color = (255, 120, 120)

    superficie_texto = font.render(texto, True, color)
    pantalla.blit(superficie_texto, (10, 10))

    pygame.display.flip()
    reloj.tick(60)
